In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline, RectBivariateSpline

# Astronomy
import astropy
from astropy.units import au, minute

from typing import List, Optional

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_pos
from asteroid_spline import get_df_shape, make_spline_df, spline_ast_data
# from asteroid_spline import make_spline_df, spline_ast_vec, spline_ast_elt
from planets_interp import get_earth_pos
from asteroid_direction import jpl_ast_dir_populate, calc_dir_ast2obs, c
from ra_dec import radec2dir
from db_utils import sp2df, df2db
import matplotlib.pyplot as plt

In [3]:
# Build table JPL.AsteroidDirections from JPl.AsteroidDirections_Import
# jpl_ast_dir_populate()

In [4]:
# Import the JPL asteroid directions
sp_name = 'JPL.GetAsteroidDirection'
params = {
    'n0': 0,
    'n1': 10,
}
dfh = sp2df(sp_name=sp_name, params=params)

sp2df(sp_name) failed!
Traceback (most recent call last):
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1628, in _execute_context
    self.dialect.do_execute(
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 663, in do_execute
    cursor.execute(statement, parameters)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/l

In [5]:
# The Horizons DataFrame
dfh

,AsteroidID,TimeID,mjd,RA,DEC,ux,uy,uz,r,rDot,delta,deltaDot,LightTime,Mag
0,1,58176000,40400.0,315.984929,-26.691893,0.642520,-0.748252,-0.165186,2.947604,0.725429,2.058383,-12.209131,17.119051,8.178
1,1,58183200,40405.0,315.339431,-27.239939,0.632400,-0.755451,-0.171357,2.949670,0.705511,2.025862,-10.291111,16.848578,8.083
2,1,58190400,40410.0,314.556310,-27.799476,0.620634,-0.763823,-0.177167,2.951679,0.685434,1.999091,-8.225469,16.625932,7.987
3,1,58197600,40415.0,313.649961,-28.358533,0.607416,-0.773133,-0.182513,2.953629,0.665205,1.978488,-6.023828,16.454580,7.893
4,1,58204800,40420.0,312.640884,-28.903941,0.593018,-0.783102,-0.187298,2.955521,0.644829,1.964400,-3.719316,16.337417,7.805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7436,1,111715200,77580.0,350.553372,-12.998581,0.961162,-0.236197,-0.142753,2.981786,-0.008784,3.710600,-13.961727,30.860116,9.498
7437,1,111722400,77585.0,352.280718,-12.408940,0.967789,-0.205834,-0.144975,2.981728,-0.031376,3.668825,-14.966463,30.512684,9.506
7438,1,111729600,77590.0,353.979618,-11.834130,0.973347,-0.175758,-0.147324,2.981605,-0.053961,3.624191,-15.941274,30.141474,9.510
7439,1,111736800,77595.0,355.648181,-11.276380,0.977868,-0.146057,-0.149805,2.981417,-0.076535,3.576793,-16.878476,29.747282,9.510


In [13]:
# Inputs
n0 = 0
n1 = 10

In [14]:
# Calculate the direction and light time
df = calc_dir_ast2obs(n0=n0, n1=n1)
mask = df.AsteroidID==1
df = df[mask]

In [15]:
df

,AsteroidID,TimeID,tAst,qAst_x,qAst_y,qAst_z,tObs,qObs_x,qObs_y,qObs_z,LightTime,ux,uy,uz
0,1,69120000,47999.983776,-1.155142,2.297488,0.282872,48000.0,-0.880033,-0.483797,-0.000050,23.362923,-0.097934,0.990084,0.100715
1,1,69125760,48003.983514,-1.192658,2.275412,0.289138,48004.0,-0.845941,-0.543194,-0.000054,23.740465,-0.121519,0.987404,0.101321
2,1,69131520,48007.983258,-1.230000,2.252623,0.295350,48008.0,-0.807882,-0.600059,-0.000056,24.108877,-0.145674,0.984069,0.101916
3,1,69137280,48011.983008,-1.267005,2.229217,0.301481,48012.0,-0.766061,-0.654113,-0.000055,24.468004,-0.170329,0.980041,0.102504
4,1,69143040,48015.982766,-1.303660,2.205197,0.307529,48016.0,-0.720707,-0.705117,-0.000054,24.817330,-0.195416,0.975287,0.103088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,90696960,62983.979958,0.388962,2.652629,0.012758,62984.0,-0.804421,-0.605790,0.000125,28.859965,0.343849,0.939018,0.003652
3747,1,90702720,62987.979812,0.347033,2.655304,0.020559,62988.0,-0.762203,-0.659471,0.000130,29.071225,0.317277,0.948315,0.005856
3748,1,90708480,62991.979676,0.305018,2.657324,0.028356,62992.0,-0.716489,-0.710085,0.000132,29.267047,0.290223,0.956925,0.008032
3749,1,90714240,62995.979550,0.262927,2.658686,0.036145,62996.0,-0.667499,-0.757430,0.000132,29.447438,0.262720,0.964818,0.010182


In [ ]:
# Get test DataFrame from JPL
